In [1]:
import numpy as np
import torch.nn as nn
import torch
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from src.helpers.functions import get_data, pc_T, predict_pca, estimate_AR_res, generate_data
from src.helpers.functions import select_AR_lag_SIC, winsor, lag_matrix
from src.helpers.refactored import ar_forecast, scale_X, reduce_dimensions, loocv_ts, standardize, forecast
from src.helpers.autoencoder import Autoencoder
from src.helpers.lstm_ae import LSTMAutoencoder
from src.helpers.forecast import Forecast

In [7]:
def out_sample(X, y, dim_method="ae", scale_method="distance_correlation", h=1, hyper_params_grid=None, forecast_method="ols",forecast_params=None, update_period=60, target="missing"):
    """ Function to perform out of sample forecasting """
    T = y.shape[0]

    M = (2000-1959)*12  # In sample periods
    N = T - M  # Out of sample periods

    forecast_spca = np.zeros((N, 1))  # Forecast errors of scaled PCA
    forecast_ar = np.zeros((N, 1))  # Forecast errors of AR model
    actual_y = np.zeros((N, 1))  # Actual values of y

    # Initialize the models
    fc = Forecast(method=forecast_method, hyper_params=forecast_params, h=h)
    ae = None

    p_max = 3  # Max number of lags for AR(p) model
    
    # Loop over all out of sample periods
    for n in range(N):
        # Print every 20 percent
        if n % (N // 5) == 0:
            print(f"Out of sample period {n} out of {N} periods")
            
        # Use all available data up to time t
        X_t = X[:(M + n), :]
        y_t = y[:M + n]
        actual_y[n] = y[M + n]

        # Standardize the data
        X_t = standardize(X_t)

        # Get number lags
        p_AR_star_n = select_AR_lag_SIC(y_t, h, p_max=p_max)

        # Compute the forecast of the AR model
        forecast_ar[n] = ar_forecast(p_AR_star_n, y_t, h)
        
        #### STEP 1: Scaling factors ####

        # Compute the betas for scaling the variables
        beta = scale_X(X_t, y_t, h, method=scale_method, p_AR_star_n=p_AR_star_n)
        
        # Winsorizing the betas
        beta_win = winsor(np.abs(beta), p=(0, 90))

        # Scale the factors by the winsorized betas
        scaleX_t = X_t * beta_win

        #### Intermezzo: Find the optimal number of factors (or other hyperparameters) ####
        
        if n == 0:
            #print("Starting hyperparameter optimization")
            hyper_params = loocv_ts(X=scaleX_t, y=y_t, h=h, p_AR_star_n=p_AR_star_n, method=dim_method, scale_method=scale_method, grid=hyper_params_grid)
            #print("Optimal Dimension Reduction hyperparameters found")
            print("-----------------------------------------------------------------")
            print("Initial hyperparameter optimization done")
            
            if dim_method == "ae":
                # Initialize the autoencoder
                ae = Autoencoder(input_dim=X.shape[1], activation=nn.SiLU, hyper_params=hyper_params)

                # Train the autoencoder on the in sample data
                ae.train_model(scaleX_t, lr=hyper_params.get("lr", 0.001), num_epochs=hyper_params.get("epochs", 300))
            elif dim_method == "lstm":
                # Initialize the autoencoder
                ae = LSTMAutoencoder(input_dim=X.shape[1], hyper_params=hyper_params)

                # Train the autoencoder on the in sample data
                ae.train_model(scaleX_t, lr=hyper_params.get("lr", 0.001), num_epochs=hyper_params.get("epochs", 300))

                print("Autoencoder training done")

        ### Updating Hyperparameters durign forecasting ###
        if n % update_period == 0 and n > 0:
            if dim_method == "ae":
                pass
            else:
                hyper_params = loocv_ts(X=X_t, y=y_t, h=h, p_AR_star_n=p_AR_star_n, method=dim_method, scale_method=scale_method, grid=hyper_params_grid)
            
        #### STEP 2: Dimension Reduction ####            

        # Compute the reduced dimensionality representation of the factors
        x_spc = reduce_dimensions(X=scaleX_t, hyper_params=hyper_params, method=dim_method, dim_red_model=ae)

        #### STEP 3: Forecasting ####

        # Add lag of y_t to the factors
        if p_AR_star_n > 0:
            x_spc = lag_matrix(x_spc, y_t, p_AR_star_n)
            y_t = y_t[p_AR_star_n-1:]
                
        # Cross validate the hyperparameters once in first period
        if n == 0 and forecast_params:
            fc.cross_validate(x_spc, y_t, hyper_params=forecast_params)
        elif n > 0 and forecast_params:
            if n % update_period == 0:
                fc.cross_validate(x_spc, y_t, hyper_params=forecast_params)

        # Compute the forecast of the PCA and scaled PCA model
        forecast_spca[n] = fc.predict(x_spc, y_t)

    # Compute the forecast errors
    error_spca = actual_y - forecast_spca
    error_ar = actual_y - forecast_ar
    
    # Compute the R squared out of sample against the AR model
    SSE_spca = np.sum(error_spca**2)
    SSE_ar = np.sum(error_ar**2)

    R2_spca = (1 - SSE_spca / SSE_ar)

    print("R2_spca: ", round(R2_spca * 100, 2))

    if dim_method == "kpca":
        dim_method += "_" + hyper_params['kernel']
        
    # Save the results to a numpy file for later use
    #np.save(f"c:/Users/Vincent/PythonProjects/Thesis/resources/results/forecasts/NOCV_{target}_{dim_method}_{scale_method}_{forecast_method}_h{h}_N{N}_pmax{p_max}_M{M}_R2{round(R2_spca * 100, 2)}.npy", forecast_spca)
    #np.save(f"c:/Users/Vincent/PythonProjects/Thesis/resources/results/errors/NOCV_{target}_{dim_method}_{scale_method}_{forecast_method}_h{h}_N{N}_pmax{p_max}_M{M}_R2{round(R2_spca * 100, 2)}.npy", error_spca)


In [3]:
variables = get_data()
X = variables['data'].values

print("Shape of X: ", X.shape)

Shape of X:  (720, 123)


In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# define the space of hyperparameters to search
space = {
    'hidden_dim': hp.choice('hidden_dim', list(np.arange(1, 30, 2))),
    'layer_dims': hp.choice('layer_dims', [[64, 32], [32, 32], [100, 50], [64, 64], [32, 32, 32], [120, 80, 40]]),
    'activation': hp.choice('activation', [nn.SiLU, nn.LeakyReLU]),
    'gauss_noise': hp.uniform('gauss_noise', 0, 1),
    'dropout': hp.uniform('dropout', 0, 0.5),
    'lr': hp.loguniform('lr', -5, -2),
    'num_epochs': hp.choice('num_epochs', [200, 300, 400]),
    'batch_size': hp.choice('batch_size', [32, 64, 100])
}

def evaluate_model(model, X_valid):
    model.eval()  # set the model to evaluation mode

    # Convert to torch tensor
    X_valid = torch.from_numpy(X_valid).float().to(model.device)

    # Reshape and run through model
    X_valid = X_valid.reshape(X_valid.shape[0], 1, X_valid.shape[1])
    output = model(X_valid)

    # Compute loss
    loss = model.loss_criterion(output, X_valid)

    return loss.item()

def objective(params):
    # load your data
    variables = get_data()
    X = variables['data'].values

    X_train, X_valid = X[:int(0.8*len(X))], X[int(0.8*len(X)):]

    # create an instance of the model
    model = Autoencoder(hyper_params=params)
    
    # train the model
    model.train_model(X_train, lr=params['lr'], num_epochs=params['num_epochs'], batch_size=params['batch_size'])

    # evaluate the model
    val_loss = evaluate_model(model, X_valid)  # you need to implement this function

    return {'loss': val_loss, 'status': STATUS_OK}

# run the optimizer
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=25, trials=trials)

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 25/25 [12:54<00:00, 31.00s/trial, best loss: 8.881653785705566]


In [12]:
print(best)

{'activation': 0, 'batch_size': 2, 'dropout': 0.44937695325960963, 'gauss_noise': 0.7412079106731622, 'hidden_dim': 0, 'layer_dims': 0, 'lr': 0.02766271599852037, 'num_epochs': 1}


In [14]:
# Set seed of numpy and torch
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# Dimension Reduction Hyperparameters
ae_params = {"hidden_dim": [ 10, 15, 20],
                "layer_dims": [[32], [64]],
                "dropout": [0.45],
                "gauss_noise": [0.2, 0.5],
                "lr": [0.03],
                "epochs": [200],
                "update_epochs": [15],
                "update_lr": [0.001]
}

rbf_params = {"gamma": 10**np.arange(-6,-2.5,.5),
            "n_components": [1, 3, 5, 7, 9, 11, 15, 20, 25, 30],
            "kernel": ["rbf"],
}

sigmoid_params = {
    "gamma": 10**np.arange(-6,-1.5,.5),
    "n_components": [1, 3, 5, 7, 9, 11, 15, 20, 25, 30],
    "kernel": ["sigmoid"],
}

pca_params = {"nfac": [1, 3, 5, 7, 9, 11, 15, 20, 25, 30]}

# Regression hyperparameters
# TODO: ENTER EXTERTATE HYPERPARAMETERS
krr_params = {
              "kernel": ["rbf"],
              "gamma": [0.3, 0.5, 0.7],
              "alpha": [0.3, 0.5, 0.7],
}

rf_params_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, None],
    "max_features": ["sqrt", "log2"],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 3],
}

# Run the forecasting exercise
for target in variables.keys():
    if target != 'inflation':
        continue
    
    print("Target: ", target)
    y = variables[target].values
    result = out_sample(
        X = X,
        y = y,
        scale_method="regression",
        dim_method="pca",
        forecast_method="ols",
        hyper_params_grid=pca_params,
        h=3,
        forecast_params=None,
        target=target,
        update_period=1000)

Target:  inflation
Out of sample period 0 out of 228 periods
Number of model configurations:  10
Best model configuration:  {'nfac': 11}
-----------------------------------------------------------------
Initial hyperparameter optimization done
Out of sample period 45 out of 228 periods
Out of sample period 90 out of 228 periods
Out of sample period 135 out of 228 periods
Out of sample period 180 out of 228 periods
Out of sample period 225 out of 228 periods
R2_spca:  -16.28


In [ ]:
errors_ar = result['error_ar']
errors_pca = result['error_pca']
errors_spca = result['error_spca']

#errors = pd.DataFrame({'errors_ar': errors_ar, 'errors_pca': errors_pca.flatten()})
#np.set_printoptions(formatter={'all':lambda x: str(x)[:7]})
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
mse_ar = np.mean(errors_ar**2)
mse_pca = np.mean(errors_pca**2)
mse_spca = np.mean(errors_spca**2)

print("MSE AR: ", mse_ar)
print("MSE PCA: ", mse_pca)
print("MSE SPCA: ", mse_spca)